# Banc d'oiseaux interactif — démonstration

Ce notebook contient une simulation **2D (projection)** d'un banc d'oiseaux (boids) et une interface interactive :

- Clique dans la figure pour placer une **cible** pour l'oiseau contrôlé.
- L'oiseau contrôlé reçoit une accélération visant la cible ; les autres boids suivent les règles de Reynolds.

Objectif : créer un *environnement* interactif prêt à être étendu (Gym env, rendu 3D, ou contrôle par RL).


In [1]:
# For mouse control 
%matplotlib qt

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
plt.rcParams['figure.figsize'] = (8,8)

class Boid:
    def __init__(self, position, velocity, bid):
        self.pos = np.array(position, dtype=np.float32)
        self.vel = np.array(velocity, dtype=np.float32)
        self.id = bid

class Flock:
    def __init__(self, n=30, bounds=20.0, max_speed=2.0, max_acc=1.0, seed=None):
        self.rng = np.random.default_rng(seed)
        self.n = n
        self.bounds = float(bounds)
        self.max_speed = float(max_speed)
        self.max_acc = float(max_acc)
        self.dt = 0.2
        self.boids = []
        for i in range(n):
            pos = (self.rng.random(2) - 0.5) * 2 * self.bounds
            vel = (self.rng.random(2) - 0.5) * 2 * self.max_speed
            self.boids.append(Boid(pos, vel, i))
        # boids params (2D)
        self.sep_radius = 2.0
        self.align_radius = 6.0
        self.coh_radius = 8.0
        self.sep_weight = 1.5
        self.align_weight = 1.0
        self.coh_weight = 1.0

    def limit(self, vec, maxval):
        mag = np.linalg.norm(vec)
        if mag > maxval and mag > 1e-8:
            return vec / mag * maxval
        return vec

    def step(self, external_actions=None):
        positions = np.array([b.pos for b in self.boids])
        velocities = np.array([b.vel for b in self.boids])
        for i, b in enumerate(self.boids):
            rel = positions - b.pos
            dist = np.linalg.norm(rel, axis=1)
            # separation
            close_mask = (dist > 0) & (dist < self.sep_radius)
            sep = np.zeros(2)
            if np.any(close_mask):
                sep = -np.sum(rel[close_mask] / (dist[close_mask][:, None]**2 + 1e-6), axis=0)
            # alignment
            align_mask = (dist > 0) & (dist < self.align_radius)
            align = np.zeros(2)
            if np.any(align_mask):
                align = np.mean(velocities[align_mask], axis=0) - b.vel
            # cohesion
            coh_mask = (dist > 0) & (dist < self.coh_radius)
            coh = np.zeros(2)
            if np.any(coh_mask):
                center = np.mean(positions[coh_mask], axis=0)
                coh = center - b.pos
            accel = (self.sep_weight * sep + self.align_weight * align + self.coh_weight * coh)
            if external_actions is not None and b.id in external_actions:
                accel = accel + external_actions[b.id]
            accel = self.limit(accel, self.max_acc)
            b.vel = b.vel + accel * self.dt
            b.vel = self.limit(b.vel, self.max_speed)
            b.pos = b.pos + b.vel * self.dt
            # bounds wrap-around
            for k in range(2):
                if b.pos[k] < -self.bounds:
                    b.pos[k] += 2 * self.bounds
                elif b.pos[k] > self.bounds:
                    b.pos[k] -= 2 * self.bounds

    def get_state(self):
        return np.array([b.pos for b in self.boids]), np.array([b.vel for b in self.boids])


In [13]:
# Interactive demo: click to set a target for the controlled boid (id = controlled_id)
from IPython.display import display

def make_interactive_demo(n=40, controlled_id=0, bounds=20.0, seed=1):
    flock = Flock(n=n, bounds=bounds, seed=seed, max_speed=4.0)
    target = np.array([0.0, 0.0], dtype=np.float32)
    fig, ax = plt.subplots()
    ax.set_xlim(-bounds, bounds)
    ax.set_ylim(-bounds, bounds)
    scat = ax.scatter([], [], s=40)
    controlled_sc = ax.scatter([], [], s=120, edgecolors='k', linewidths=1.2)
    target_sc = ax.scatter([], [], s=80, marker='x')
    txt = ax.text(0.02, 0.98, '', transform=ax.transAxes, va='top')

    def on_click(event):
        # only accept clicks inside axes
        if event.inaxes != ax:
            return
        target[0] = event.xdata
        target[1] = event.ydata
        # update target marker immediately
        target_sc.set_offsets([target])
        fig.canvas.draw_idle()

    fig.canvas.mpl_connect('button_press_event', on_click)

    def update(frame):
        print("update", frame)
        # controlled boid receives accel towards target
        positions, velocities = flock.get_state()
        my_pos = positions[controlled_id]
        
        # compute desired acceleration for controlled boid
        desired = target - my_pos
        if np.linalg.norm(desired) > 1e-6:
            desired = desired / np.linalg.norm(desired) * flock.max_acc
        else:
            desired = np.zeros(2)
            
        actions = {controlled_id: desired}
        
        # 🟢 step updates *all* boids, not just controlled one
        flock.step(external_actions=actions)
        
        # update display
        positions, _ = flock.get_state()
        scat.set_offsets(positions)
        controlled_sc.set_offsets([positions[controlled_id]])
        target_sc.set_offsets([target])
        txt.set_text(f'Tick: {frame} | Boids: {len(flock.boids)}')
        
        return scat, controlled_sc, target_sc, txt


    # initialize target to controlled boid's initial pos
    init_pos, _ = flock.get_state()
    target[:] = init_pos[controlled_id].copy()
    target_sc.set_offsets([target])
    positions, _ = flock.get_state()
    scat.set_offsets(positions)
    controlled_sc.set_offsets([positions[controlled_id]])
    ani = animation.FuncAnimation(
        fig, update,
        frames=np.arange(10000),   # un grand nombre d’itérations
        interval=100,              # en ms → 0.1 s
        blit=False,
        repeat=True)
    plt.show()
    return flock

# To run the demo in this notebook, call make_interactive_demo()
print('Call make_interactive_demo() to start the interactive demo. Ex: make_interactive_demo(n=40, controlled_id=0)')


Call make_interactive_demo() to start the interactive demo. Ex: make_interactive_demo(n=40, controlled_id=0)


In [19]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import time

plt.ion()  # mode interactif

class Boid:
    def __init__(self, position, velocity, bid):
        self.pos = np.array(position, dtype=np.float32)
        self.vel = np.array(velocity, dtype=np.float32)
        self.id = bid

class Flock:
    def __init__(self, n=40, bounds=20.0, seed=None):
        rng = np.random.default_rng(seed)
        self.boids = [Boid((rng.random(2)-0.5)*2*bounds, (rng.random(2)-0.5)*4, i) for i in range(n)]
        self.bounds = bounds
        self.dt = 0.1
        self.sep_radius = 2.0
        self.align_radius = 6.0
        self.coh_radius = 8.0
        self.sep_weight = 1.5
        self.align_weight = 1.0
        self.coh_weight = 1.0
        self.max_speed = 4.0
        self.max_acc = 2.0

    def limit(self, vec, maxval):
        mag = np.linalg.norm(vec)
        if mag > maxval and mag > 1e-8:
            return vec / mag * maxval
        return vec

    def step(self, external_actions=None):
        positions = np.array([b.pos for b in self.boids])
        velocities = np.array([b.vel for b in self.boids])
        for b in self.boids:
            rel = positions - b.pos
            dist = np.linalg.norm(rel, axis=1) + 1e-9
            # masks
            sep_mask = (dist>0) & (dist < self.sep_radius)
            align_mask = (dist>0) & (dist < self.align_radius)
            coh_mask = (dist>0) & (dist < self.coh_radius)
            sep = np.zeros(2)
            if np.any(sep_mask):
                sep = -np.sum(rel[sep_mask] / (dist[sep_mask][:,None]**2 + 1e-6), axis=0)
            align = np.zeros(2)
            if np.any(align_mask):
                align = np.mean(velocities[align_mask], axis=0) - b.vel
            coh = np.zeros(2)
            if np.any(coh_mask):
                center = np.mean(positions[coh_mask], axis=0)
                coh = center - b.pos
            accel = self.sep_weight*sep + self.align_weight*align + self.coh_weight*coh
            if external_actions and b.id in external_actions:
                accel += external_actions[b.id]
            accel = self.limit(accel, self.max_acc)
            b.vel = self.limit(b.vel + accel*self.dt, self.max_speed)
            b.pos = b.pos + b.vel * self.dt
            # wrap
            for k in range(2):
                if b.pos[k] < -self.bounds: b.pos[k] += 2*self.bounds
                if b.pos[k] >  self.bounds: b.pos[k] -= 2*self.bounds

    def get_state(self):
        return np.array([b.pos for b in self.boids]), np.array([b.vel for b in self.boids])

# --- setup GUI ---
flock = Flock(n=40, bounds=20.0, seed=2)
controlled_id = 0
target = flock.get_state()[0][controlled_id].copy()

fig, ax = plt.subplots()
ax.set_xlim(-flock.bounds, flock.bounds)
ax.set_ylim(-flock.bounds, flock.bounds)
scat = ax.scatter([], [], s=40)
controlled_sc = ax.scatter([], [], s=120, edgecolors='k', linewidths=1.2)
target_sc = ax.scatter([target[0]], [target[1]], s=80, marker='x')
txt = ax.text(0.02, 0.98, '', transform=ax.transAxes, va='top')

def on_click(event):
    if event.inaxes != ax: 
        return
    target[0] = event.xdata
    target[1] = event.ydata
    target_sc.set_offsets([target])
    # immediate redraw
    fig.canvas.draw_idle()

fig.canvas.mpl_connect('button_press_event', on_click)

# initialize
positions, _ = flock.get_state()
scat.set_offsets(positions)
controlled_sc.set_offsets([positions[controlled_id]])
fig.canvas.draw()
fig.canvas.flush_events()

# --- manual animation loop (robuste) ---
interval = 0.1  # secondes (0.1s)
frame = 0
try:
    while True:
        # compute action for controlled boid
        positions, velocities = flock.get_state()
        my_pos = positions[controlled_id]
        desired = target - my_pos
        if np.linalg.norm(desired) > 1e-6:
            desired = desired / np.linalg.norm(desired) * flock.max_acc
        else:
            desired = np.zeros(2)
        actions = {controlled_id: desired}

        # update physics
        flock.step(external_actions=actions)

        # update plot
        positions, _ = flock.get_state()
        scat.set_offsets(positions)
        controlled_sc.set_offsets([positions[controlled_id]])
        target_sc.set_offsets([target])
        txt.set_text(f'Frame: {frame}')
        # force redraw
        fig.canvas.draw()
        fig.canvas.flush_events()

        frame += 1
        time.sleep(interval)  # pause exactly ~interval seconds (can adjust)
except KeyboardInterrupt:
    print("Animation stopped par l'utilisateur (Ctrl-C).")


Animation stopped par l'utilisateur (Ctrl-C).


## Utilisation

1. Exécute la cellule ci-dessus pour lancer la démo interactive (la figure s'ouvrira dans le notebook).  
2. Clique dans la figure pour déplacer la **cible** ; l'oiseau contrôlé (marker plus grand) essaiera d'aller vers la cible.

Tu peux modifier les paramètres `n` (nombre d'oiseaux) et `controlled_id` avant d'exécuter.


In [15]:
# Lance la démo interactive (décommente pour exécuter ici)
flock = make_interactive_demo(n=40, controlled_id=0, bounds=20.0, seed=2)
print('Pour lancer la simulation, exécute : flock = make_interactive_demo(n=40, controlled_id=0)')


Pour lancer la simulation, exécute : flock = make_interactive_demo(n=40, controlled_id=0)
